In [2]:
import pandas as pd
from sklearn import datasets
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
import mlflow
from mlflow.models import infer_signature

In [3]:
##set tracking uri

mlflow.set_tracking_uri("https://127.0.0.1:5000")


In [19]:
##loading data set - petal length, petal width, sepal length, sepal length, sepal width - flowers

X,y = datasets.load_iris(return_X_y= True)

#split data into train and test sets 

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20)

#define model hyperparameter
params = {"penalty":"l2","solver" : "newton-cg", "max_iter":1000, "multi_class":"auto", "random_state":8888}

##train model
lr = LogisticRegression(**params)
lr.fit(X_train, y_train)

c:\git\Projects\MLFlowProjects\venv\lib\site-packages\sklearn\linear_model\_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


LogisticRegression(max_iter=1000, multi_class='auto', random_state=8888,
                   solver='newton-cg')

In [20]:
##Prediction on the test set
y_pred = lr.predict(X_test)

In [21]:
accuracy = accuracy_score(y_test, y_pred)

In [22]:
###MLFlow Tracking
mlflow.set_tracking_uri("http://127.0.0.1:5000")

#create ML Flow experiment
mlflow.set_experiment("Quick Start MLFlow")

##start MLFLow Run
with mlflow.start_run():
    ##log hyperparmaters
    mlflow.log_params(params)

    ##log accuracy metrics
    mlflow.log_metrics({"accuracy":accuracy})

    ##tag for the run to remind what is this run for
    mlflow.set_tag("Training Infor", "Basic LR model for IRIS data")

    ##Infer the model signature
    signature = infer_signature(X_train, lr.predict(X_train))

    ##log the model
    model_info = mlflow.sklearn.log_model(
        sk_model = lr,
        artifact_path = "iris_model",
        signature = signature,
        input_example = X_train,
        registered_model_name="tracking-quickstart"


    )





Registered model 'tracking-quickstart' already exists. Creating a new version of this model...
2025/05/13 07:48:08 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: tracking-quickstart, version 3


🏃 View run victorious-rat-652 at: http://127.0.0.1:5000/#/experiments/934240718161479422/runs/e0e501f71a1949f1b729d6b1d50e6f80
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/934240718161479422


Created version '3' of model 'tracking-quickstart'.


In [24]:
model_info.model_uri

'runs:/e0e501f71a1949f1b729d6b1d50e6f80/iris_model'

In [27]:
##Inferencing And validating my model

import mlflow
from mlflow.models import Model

model_uri = model_info.model_uri
# The model is logged with an input example
pyfunc_model = mlflow.pyfunc.load_model(model_uri)
input_data = pyfunc_model.input_example

# Verify the model with the provided input data using the logged dependencies.
# For more details, refer to:
# https://mlflow.org/docs/latest/models.html#validate-models-before-deployment
mlflow.models.predict(
    model_uri=model_uri,
    input_data=input_data,
    env_manager="uv",
)

2025/05/13 07:56:46 INFO mlflow.models.flavor_backend_registry: Selected backend for flavor 'python_function'
2025/05/13 07:56:46 INFO mlflow.utils.virtualenv: Creating a new environment in C:\Users\prane\AppData\Local\Temp\tmpyzwqf60w\envs\virtualenv_envs\mlflow-2039b1bcc5e02270a0cf33e5166e26b2edb5c332 with python version 3.10.16 using uv
2025/05/13 07:56:49 INFO mlflow.utils.virtualenv: Installing dependencies
2025/05/13 07:56:49 WARNING mlflow.utils.virtualenv: Failed to symlink model directory during dependency installation Copying instead. Exception: [WinError 1314] A required privilege is not held by the client: 'C:\\Users\\prane\\AppData\\Local\\Temp\\tmpsborlv05\\iris_model\\conda.yaml' -> 'C:\\Users\\prane\\AppData\\Local\\Temp\\tmp_gilr_lg\\conda.yaml'
2025/05/13 07:56:59 WARNING mlflow.utils.virtualenv: Failed to symlink model directory during dependency installation Copying instead. Exception: [WinError 1314] A required privilege is not held by the client: 'C:\\Users\\prane

MlflowException: Non-zero exit code: 1
Command: ['cmd', '/c', "C:\\Users\\prane\\AppData\\Local\\Temp\\tmpyzwqf60w\\envs\\virtualenv_envs\\mlflow-2039b1bcc5e02270a0cf33e5166e26b2edb5c332\\Scripts\\activate.bat & python c:\\git\\Projects\\MLFlowProjects\\venv\\lib\\site-packages\\mlflow\\pyfunc\\_mlflow_pyfunc_backend_predict.py --model-uri file:///C:/Users/prane/AppData/Local/Temp/tmpsborlv05/iris_model --content-type json --input-path 'C:\\Users\\prane\\AppData\\Local\\Temp\\tmpy7_tbyub\\input.json'"]

An exception occurred while running model prediction within a uv environment. You can find the error message from the prediction subprocess by scrolling above.

In [28]:
## load the model back for prediction as a generic python function model
loaded_model = mlflow.pyfunc.load_model(model_info.model_uri)
predictions = loaded_model.predict(X_test)

iris_features_name = datasets.load_iris().feature_names

result = pd.DataFrame(X_test, columns=iris_features_name)
result['actual_class'] = y_test
result['predicted_class'] = predictions


In [29]:
result

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),actual_class,predicted_class
0,6.0,2.7,5.1,1.6,1,2
1,5.0,2.0,3.5,1.0,1,1
2,6.5,3.0,5.8,2.2,2,2
3,6.4,2.7,5.3,1.9,2,2
4,6.6,3.0,4.4,1.4,1,1
5,6.2,2.8,4.8,1.8,2,2
6,4.3,3.0,1.1,0.1,0,0
7,5.8,4.0,1.2,0.2,0,0
8,6.3,2.5,4.9,1.5,1,1
9,5.5,2.6,4.4,1.2,1,1


Model Registry 

In [30]:
###MLFlow Tracking
mlflow.set_tracking_uri("http://127.0.0.1:5000")

#create ML Flow experiment
mlflow.set_experiment("Quick Start MLFlow")

##start MLFLow Run
with mlflow.start_run():
    ##log hyperparmaters
    mlflow.log_params(params)

    ##log accuracy metrics
    mlflow.log_metrics({"accuracy":1.0})

    ##tag for the run to remind what is this run for
    mlflow.set_tag("Training Info2", "Basic LR model for IRIS data")

    ##Infer the model signature
    signature = infer_signature(X_train, lr.predict(X_train))

    ##log the model
    model_info = mlflow.sklearn.log_model(
        sk_model = lr,
        artifact_path = "iris_model",
        signature = signature,
        input_example = X_train,


    )





🏃 View run funny-dolphin-224 at: http://127.0.0.1:5000/#/experiments/934240718161479422/runs/83750ab581f44b17af6f60c375854d92
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/934240718161479422


In [ ]:
##inferencing from model from model registry

import mlflow.sklearn
model_name ="tracking-quickstart"

model_version = "latest"

model_uri = f"models:/{model_name}/{model_version}"